# Action Client node with Jupyter 

### Import

In [1]:
import rospy
import assignment_2_2023.msg
from assignment_2_2023.msg import GoalCoord
from assignment_2_2023.srv import CancelGoal, CancelGoalResponse
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math

### New Imports

In [2]:
%matplotlib widget
#import jupyros as jr
#from jupyros import ros3d
import os
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

### Init of the node

In [3]:
time.sleep(1)

# Initialize the node
rospy.init_node('user_interface')



# Goal insert by the user publisher
pubgoal = rospy.Publisher('/goal_coord', GoalCoord, queue_size=1)

# wait for the service to be available
rospy.wait_for_service('/cancel_goal')
cancgoalServ = rospy.ServiceProxy('/cancel_goal', CancelGoal)

# Keep the node running
#rospy.spin()

welcText = widgets.Textarea(value='Hi, welcome to the robot planner simulation!!', layout=widgets.Layout(width='auto'))
display(welcText)

goalSet = []
goalCanc = []
# response = cancel_goal_service() to call it 

Textarea(value='Hi, welcome to the robot planner simulation!!', layout=Layout(width='auto'))

### Init of the user Interface
Starting the interaction with the user throuh widgets to let him set the goal

In [4]:
introText = widgets.Textarea(value='Please choose the goal position you want to reach with the robot.', 
                             layout=widgets.Layout(width='auto'))
display(introText)
xdes = widgets.Text(value='Choose the x coordinate:')
xgoal = widgets.IntSlider(
    value = 0,
    min = -8,
    max = 8,
    step = 1,
    layout=widgets.Layout(width='auto')
)
display(xdes, xgoal)
ydes = widgets.Text(value='Choose the y coordinate:')
ygoal = widgets.IntSlider(
    value = 0,
    min = -8,
    max = 8,
    step = 1,
    layout=widgets.Layout(width='auto')
)
display(ydes, ygoal)
bttgoal = widgets.Button(description='Send goal', 
                         layout=widgets.Layout(width='auto', display='flex', justify_content='center'), 
                         style=widgets.ButtonStyle(button_color='red', font_weight='bold', color='white'))
box = widgets.Box([bttgoal])
box.layout = widgets.Layout(display='flex', justify_content='center', width='100%')
out = widgets.Output()
display(box, out)

# Function to send the goal to the robot (modifica i print)
def on_button_clicked_set(b):
    with out:
        if xgoal.value == None or ygoal.value == None:
            print('Please insert a valid number')
        else:
            goal = GoalCoord()
            goal.coord_x = xgoal.value
            goal.coord_y = ygoal.value
            pubgoal.publish(goal)
            goalSet.append([xgoal.value, ygoal.value])
            print('Goal sent to the robot')

bttgoal.on_click(on_button_clicked_set)


Textarea(value='Please choose the goal position you want to reach with the robot.', layout=Layout(width='auto'…

Text(value='Choose the x coordinate:')

IntSlider(value=0, layout=Layout(width='auto'), max=8, min=-8)

Text(value='Choose the y coordinate:')

IntSlider(value=0, layout=Layout(width='auto'), max=8, min=-8)

Box(children=(Button(description='Send goal', layout=Layout(display='flex', justify_content='center', width='a…

Output()

Interaction with the user throuh widgets to let him cancel the goal

Check the service and the serviceproxy and understand how to do it to have the return string to know the status of the request and send it back

In [5]:
cancText = widgets.Textarea(value='Here you can cancel the the latest goal', 
                             layout=widgets.Layout(width='auto'))
display(cancText)
bttcanc = widgets.Button(description='Cancel goal', 
                         layout=widgets.Layout(width='auto', display='flex', justify_content='center'), 
                         style=widgets.ButtonStyle(button_color='purple', font_weight='bold', color='black'))
boxcanc = widgets.Box([bttcanc])
boxcanc.layout = widgets.Layout(display='flex', justify_content='center', width='100%')
out = widgets.Output()
display(boxcanc, out)

def on_button_clicked_canc(b):
    with out:
        if len(goalSet) == 0:
            print('No goal to cancel')
        else:
            status = cancgoalServ()
            # perform the control
            print("status = ", status.stat)
            if (status.stat == "Done"):
                goalCanc.append(goalSet[-1])
                print('Goal cancelled')
            elif (status.stat == "Reached"):
                print('Goal not cancelled becouse already reached')
            else:
                print('Goal not cancelled, errror occoured')


bttcanc.on_click(on_button_clicked_canc)


Textarea(value='Here you can cancel the the latest goal', layout=Layout(width='auto'))

Box(children=(Button(description='Cancel goal', layout=Layout(display='flex', justify_content='center', width=…

Output()